In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

Chain of Density: https://arxiv.org/pdf/2309.04269.pdf
```
RAG에서의 Chain of Density:
RAG 시스템에서 CoD는 검색된 문서나 지식을 더 효과적으로 활용하여 높은 품질의 
응답을 생성하는 데 사용될 수 있습니다.

목적:

검색된 정보를 더 밀도 있게 활용
응답의 정보 밀도를 높이면서도 간결성 유지

RAG에서의 적용:
a. 초기 응답 생성: 검색된 문서를 바탕으로 기본적인 응답 생성
b. 밀도 증가: 추가 정보나 누락된 중요 개념을 점진적으로 통합
c. 재구성: 응답을 재구성하여 더 풍부하고 정확한 정보 제공

장점:

더 포괄적이고 정확한 응답 생성
검색된 문서의 정보를 더 효율적으로 활용
불필요한 반복이나 중복 정보 감소

구현 방법:

다단계 프롬프팅: LLM에 여러 단계의 프롬프트를 제공하여 응답을 점진적으로 개선
반복적 검색: 초기 응답을 바탕으로 추가 정보 검색 및 통합

예시 과정:
a. 초기 응답 생성
b. 누락된 중요 정보 식별
c. 추가 정보 검색 또는 기존 검색 결과에서 추출
d. 새로운 정보를 통합하여 응답 개선
e. 필요시 이 과정을 반복

도전 과제:

적절한 밀도 수준 결정
응답의 일관성 유지
계산 비용과 시간 관리

RAG 시스템 개선:
CoD를 통해 RAG 시스템은 단순히 정보를 검색하고 요약하는 것을 넘어, 
더 깊이 있고 맥락에 맞는 응답을 생성할 수 있습니다.

Chain of Density를 RAG에 적용함으로써, 시스템은 더 정교하고 정보가 
풍부한 응답을 생성할 수 있습니다. 이는 특히 복잡한 질문이나 다양한 
정보 소스를 필요로 하는 상황에서 유용할 수 있습니다. 
그러나 이 과정이 응답 생성 시간을 늘릴 수 있으므로, 
실시간 응답이 필요한 경우에는 적절한 균형을 찾는 것이 중요합니다.
```

In [2]:
import time
import textwrap

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough

# Load some data to summarize
loader = WebBaseLoader("https://teddylee777.github.io/data-science/optuna/")
docs = loader.load()
content = docs[0].page_content

# Get this prompt template
# https://smith.langchain.com/hub/lawwu/chain_of_density?organizationId=4238ea05-3ccc-5451-bdf6-3935d7cf64b8
prompt = hub.pull("lawwu/chain_of_density")

# The chat model output is a JSON list of dicts, with SimpleJsonOutputParser
# we can convert it o a dict, and it suppors streaming.
json_parser = SimpleJsonOutputParser()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
chain = (
    {"ARTICLE": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.1)
    | json_parser
)

In [4]:
chain.invoke(content)

[{'Missing_Entities': 'Optuna',
  'Denser_Summary': 'This article discusses Optuna, a hyperparameter optimization library for machine learning. Optuna provides a simple and efficient way to search for the best hyperparameters for a given model. It uses a variety of search algorithms to explore the hyperparameter space and find the optimal values. With Optuna, you can easily tune the hyperparameters of your machine learning models and improve their performance.'},
 {'Missing_Entities': 'trial.suggest_categorical()',
  'Denser_Summary': 'Optuna provides a suggest_categorical() function that allows you to suggest categorical hyperparameters for optimization. This function takes the name of the hyperparameter and a list of choices as input. Optuna will then explore different combinations of these choices to find the best value for the hyperparameter. By using suggest_categorical(), you can easily optimize categorical hyperparameters and improve the performance of your machine learning mode

In [6]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
import json


# Load some data to summarize
loader = WebBaseLoader(
    "https://www.aitimes.com/news/articleView.html?idxno=131777")
docs = loader.load()
content = docs[0].page_content
# Load the prompt
# prompt = hub.pull("langchain-ai/chain-of-density:4f55305e")


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)


prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (50 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)


# Create the chain, including
chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-4o",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)

```json
[
    {
        "Missing_Entities": "박정현 서울대 EPM 연구원; 데이터사이언스, 머신러닝, 인공지능 정의; 데이터사이언스 대학원 및 인공지능 대학원 설립; '인공지능 국가전략' 발표",
        "Denser_Summary": "이 기사는 데이터사이언스, 머신러닝, 인공지능에 대한 기본적인 이해와 이 분야들의 차이점을 설명합니다. 박정현 서울대 EPM 연구원은 이 분야들이 실생활에 어떻게 적용되고 있는지, 그리고 이 분야에서 일하고자 하는 사람들을 위한 조언을 제공합니다. 또한, 데이터사이언스 대학원 및 인공지능 대학원의 설립과 '인공지능 국가전략' 발표와 같은 교육 및 정책적 변화에 대해서도 언급합니다. 이 글은 데이터사이언스, 머신러닝, 인공지능의 정의를 포함하여 이 분야의 기초적인 지식을 제공하고자 합니다."
    },
    {
        "Missing_Entities": "데이터사이언스의 이론적, 실용적 측면; 튜링테스트; 지도학습, 비지도 학습",
        "Denser_Summary": "박정현 서울대 EPM 연구원은 데이터사이언스, 머신러닝, 인공지능의 정의와 실용적 적용을 설명하며, 데이터사이언스 대학원과 '인공지능 국가전략' 발표를 언급합니다. 데이터사이언스는 이론적, 실용적 측면으로 나뉘며, 인공지능의 이해를 위해 튜링테스트를, 머신러닝은 지도학습과 비지도 학습으로 구분하여 설명합니다. 이 글은 이 분야에 대한 깊은 이해를 돕고, 관련 교육 및 정책 변화에 대해 논합니다."
    },
    {
        "Missing_Entities": "알파고와 이세돌 대결; OpenAI의 GPT-3 공개; 머신러닝의 실생활 적용 예",
        "Denser_Summary": "박정현 서울대 EPM 연구원은 데이터사이언스, 머신러닝, 인공지능의 기본 개념과 이론적, 실용적 측면을 소개합니다. 알파고와 이세돌의 대결, OpenAI의 GPT-3 공개와 같은 사례를 통해 이 기술들